##Collaborative Filtering : Debashish Malakar

In [0]:
import pandas as pd
import numpy as np

In [0]:
df = pd.read_csv('views.csv')
df2 = pd.read_csv('users.csv',index_col = '_id')
df1 = pd.read_csv('posts.csv',index_col = '_id')
#df.head()


In [48]:
z = np.array(df['timestamp'])
ts = [1 for i in z]
df.insert(3,"ts",ts)
df.head()

,user_id,post_id,timestamp,ts
0,5df49b32cc709107827fb3c7,5ec821ddec493f4a2655889e,2020-06-01T10:46:45.131Z,1
1,5ed3748576027d35905ccaab,5ed4cbadbd514d602c1531a6,2020-06-01T09:39:20.021Z,1
2,5ed0defa76027d35905cc2de,5eac305f10426255a7aa9dd3,2020-06-01T08:12:42.682Z,1
3,5ed0defa76027d35905cc2de,5ed1ff0276027d35905cc60d,2020-06-01T08:10:23.880Z,1
4,5ed0defa76027d35905cc2de,5ed3820f76027d35905ccac8,2020-06-01T08:08:54.124Z,1


In [49]:
rec = df.pivot_table(index= 'post_id',columns='user_id',values = 'ts').fillna(0)
rec.head()

user_id,5d60098a653a331687083238,5d610ae1653a331687083239,5d618359fc5fcf3bdd9a0910,5d6d2bb87fa40e1417a49315,5d7c994d5720533e15c3b1e9,5d90fc1a641ef364939fb6cc,5d9a2982979d5962253c2f81,5de50d768eab6401affbb135,5ded443342a8854bf6eaba9a,5deeef6142a8854bf6eabab9,5defd51362624b0135ea9fd2,5df11ee1ee4bb5252b4f5344,5df20f1fee4bb5252b4f5351,5df24eeaee4bb5252b4f5366,5df3f8f2ee4bb5252b4f5393,5df49b32cc709107827fb3c7,5e0d5ce12a37d20505da2a5d,5e1ef04c2a37d20505da2b8b,5e26bb482a37d20505da2c10,5e3563348d344822fed4d13a,5e35a5ed8d344822fed4d13e,5e365e758d344822fed4d144,5e37b16ceab55d319938a6f8,5e4195c3eab55d319938a82c,5e4ce251f5561b1994c8e40d,5e5855ced701ab08af792b51,5e5af599d701ab08af792b63,5e5b4511d701ab08af792b6d,5e5dfbbefbc8805f69e02c91,5e783375cfc8b713f5ac7c5b,5e78ce84cfc8b713f5ac7cee,5e7b30a6cfc8b713f5ac7d95,5e7cf05bcfc8b713f5ac7db7,5e7dde87a3258347b42f2108,5e7de336a3258347b42f210e,5e7def80a3258347b42f2124,5e7f4ad1a3258347b42f2155,5e822b86a3258347b42f2360,5e8309c4a3258347b42f23f4,5e840a75a3258347b42f2437,...,5eb4f73910426255a7aaa0cc,5eb508ae10426255a7aaa12c,5eb7aeccfd92f539c465ddd7,5ebc2ec4514aab59896bcc84,5ebcf2f2514aab59896bcd00,5ec11abc74f7660d73aa0fbd,5ec2204374f7660d73aa100f,5ec3ba5374f7660d73aa1201,5ec553e2f2781131cc7e5138,5ecb979eeaff6b0c3a58a4f0,5ecd5a5c7023451e662235c1,5ecdf7397023451e66223784,5ece02427023451e66223861,5ecf6e6576027d35905cbeec,5ecfa94676027d35905cbf78,5ecfaa0076027d35905cbf83,5ecfbcb376027d35905cbfe5,5ecfdaa976027d35905cc02c,5ed09e8576027d35905cc164,5ed0bfc176027d35905cc1f8,5ed0cfc676027d35905cc26a,5ed0d9bc76027d35905cc2c2,5ed0defa76027d35905cc2de,5ed0dfec76027d35905cc2ec,5ed0fc0976027d35905cc448,5ed12ae476027d35905cc4a1,5ed13bb076027d35905cc4ba,5ed142cd76027d35905cc4cd,5ed232b976027d35905cc689,5ed237ee76027d35905cc6c5,5ed2399b76027d35905cc70f,5ed24d5a76027d35905cc7d7,5ed2672576027d35905cc7f6,5ed2edf176027d35905cc961,5ed350ed76027d35905cca2c,5ed35aa376027d35905cca67,5ed3737976027d35905ccaa2,5ed3748576027d35905ccaab,5ed3cbcbbd514d602c1530a7,5ed3e898bd514d602c1530de
post_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5d62abaa65218653a132c956,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5d6d39567fa40e1417a4931c,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5d7d23315720533e15c3b1ee,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5d7d405e5720533e15c3b1f3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5d80dfbc6c53455f896e600e,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
from scipy.sparse import csr_matrix
rec_matrix = csr_matrix(rec.values)
from sklearn.neighbors import NearestNeighbors


In [51]:
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(rec_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [0]:
def post_id(query_index):
  distances, indices = model_knn.kneighbors(rec.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)
  x= []
  for i in range(0, len(distances.flatten())):
    x.append(rec.index[indices.flatten()[i]])

  x.remove(x[0])
  return(x)

In [0]:
def post_recommend(df1,rec,query_index,x):
  print('Recommendations for post id {0}'.format(rec.index[query_index]))
  print(df1.loc[rec.index[query_index]].head())
  print("*************************************************************************************")
  df1 = pd.read_csv('posts.csv',index_col = '_id')
  return df1.loc[x]  

In [0]:
rec2 = df.pivot_table(index= 'user_id',columns='post_id',values = 'ts').fillna(0)
rec2_matrix = csr_matrix(rec2.values)

In [55]:
model_knn1 = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn1.fit(rec2_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [0]:
def user_id(query_index2):
  distances2, indices2 = model_knn1.kneighbors(rec2.iloc[query_index2,:].values.reshape(1, -1), n_neighbors = 6)
  y= []
  for i in range(0, len(distances2.flatten())):
    y.append(rec2.index[indices2.flatten()[i]])

  y.remove(y[0])
  return y

In [0]:
def user_post_id(df,rec2,query_index2,y):
  post_ = []
  for i in y:
    post_list = []
    post = df.loc[df['user_id'] == i]
    post2 = post['post_id']
    a = np.array(post2)
    post_.append(a)
  user = df.loc[df['user_id'] ==rec2.index[query_index2]]
  user_post = user['post_id']
  b = np.array(user_post)
  c = []
  for j in post_:
    for i in j:
      if i in b:
        continue
      else:
        c.append(i)
  c = c[0:5]
  return c

In [0]:
def user_recommend(df2,df1,rec2,c,query_index2):
  print('Recommendations for user id {0}'.format(rec2.index[query_index2]))
  print(df2.loc[rec2.index[query_index2]].head())
  print("*************************************************************************************")
  #df1 = pd.read_csv('posts.csv',index_col = '_id')
  return df1.loc[c]  

##Post-based Recommendation

In [59]:
post = input('Enter post_id : ')
count1 = 0
for i in rec.index:
  if i == post:
    break
  else:
    count1+=1
query_index = count1
rec.index[query_index]
post_recommend(df1,rec,query_index,post_id(query_index))

Enter post_id : 5ed1ff0276027d35905cc60d
Recommendations for post id 5ed1ff0276027d35905cc60d
title         AWS services and how to launch OS on AWS Cloud
category                 Computer Technology|Cloud Computing
 post_type                                              blog
Name: 5ed1ff0276027d35905cc60d, dtype: object
*************************************************************************************


,title,category,post_type
_id,,,
5ecf96e876027d35905cbf46,GAN's Part(2),Computer Technology|Machine Learning,blog
5ecf23b87023451e66223944,Keep working hard !!,Photography,artwork
5ed3820f76027d35905ccac8,Configure Docker with Django; PostgreSQL; Pg-a...,Computer Technology|Computer Application|Infor...,blog
5ecfa0ca76027d35905cbf57,Recommend Systems Machine Learning,Computer Technology|Machine Learning,blog
5ed0e20776027d35905cc2fe,Here are 7 of my favourite books that I've rea...,Literature|Stories|Fictions|Movements in Liter...,blog


##User-based Recommendation

In [60]:
user = input('Enter user_id : ')
count2 = 0
for i in rec2.index:
  if i == user:
    break
  else:
    count2+=1
query_index2 = count2
rec2.index[query_index2]
user_recommend(df2,df1,rec2,user_post_id(df,rec2,query_index2,user_id(query_index2)),query_index2)

Enter user_id : 5d9a2982979d5962253c2f81
Recommendations for user id 5d9a2982979d5962253c2f81
name         Nischaya Sharma
gender                  male
academics      undergraduate
Name: 5d9a2982979d5962253c2f81, dtype: object
*************************************************************************************


,title,category,post_type
_id,,,
5ed13d2876027d35905cc4c2,Launching an EC2 instance.,Technology,skill
5ecce8a5eaff6b0c3a58a5e9,The Waves Have Stood Still,Photography,artwork
5ecd5d417023451e662235c5,"Machine Learning”&“Operations"" (MlOps)",Computer Technology|Machine Learning,blog
5ecc186aeaff6b0c3a58a53f,Beliefs ❣️,Visual Arts|Photography,artwork
5ecc1d4eeaff6b0c3a58a565,Faith in yourself,Photography,artwork
